In [10]:
import os
import src.TriLinear as TriLinear
from TSB_AD.evaluation.metrics import get_metrics
import torch
import pandas as pd

from src.utils import create_sequences, anomaly_scores
import torch.nn as nn
import torch.optim as optim

In [11]:
# Check device
print("CUDA available: ", torch.cuda.is_available())
print("cuDNN version: ", torch.backends.cudnn.version())
if torch.cuda.is_available():
    print("GPU device count: ", torch.cuda.device_count())
    print("GPU device name: ", torch.cuda.get_device_name(0))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA available:  True
cuDNN version:  90100
GPU device count:  1
GPU device name:  NVIDIA GeForce RTX 2060


In [12]:
dataset_folder = os.path.join('..', 'datasets', 'light_curves')
mdwarfs_filename   = '14_mdwarfs.csv'

# Mdwarfs
rising_start_time = -20       # start of rising phase
flare_end_time    = 30        # end of full flare duration


df = pd.read_csv(os.path.join(dataset_folder, mdwarfs_filename))

# Build true-label masks
df['label_rising'] = (
    (df['timestamps'] >= rising_start_time) & (df['timestamps'] <= 0)
).astype(int)
df['label_full'] = (
    (df['timestamps'] >= rising_start_time) & (df['timestamps'] <= flare_end_time)
).astype(int)


mdwarfs_light_curve = df['nor'].astype(float).values.reshape(-1, 1)
true_rising_labels  = df['label_rising'].to_numpy()
true_full_labels    = df['label_full'].to_numpy()

timestamps_array = df['timestamps'].to_numpy()
idx_training = round(len(timestamps_array)*0.3)


In [13]:
## Linear model part
seq_length = 125
horizon_size = 50

num_epochs = 100
learning_rate = 0.005

X_train, Y_train = create_sequences(mdwarfs_light_curve[:idx_training].squeeze(axis=-1), seq_length, horizon_size)
X_test, Y_test = create_sequences(mdwarfs_light_curve[idx_training - seq_length - horizon_size + 1:].squeeze(axis=-1),
                                          seq_length, horizon_size)


In [14]:
# 2. Move model to GPU
tri_model = TriLinear.Model(seq_length, horizon_size).to(device)

X_train = torch.tensor(X_train).float().to(device)
Y_train = torch.tensor(Y_train).float().to(device)
X_test = torch.tensor(X_test).float().to(device)
Y_test = torch.tensor(Y_test).float().to(device)

# 3. Train model with data on GPU
criterion = nn.MSELoss()
optimizer = optim.Adam(tri_model.parameters(), lr=learning_rate)

# Variables to store losses
train_losses = []
test_losses = []

# Training loop
for epoch in range(num_epochs):
    tri_model.train()

    optimizer.zero_grad()
    outputs = tri_model(X_train.unsqueeze(-1))

    # Training loss
    train_loss = criterion(outputs.squeeze(), Y_train)
    train_loss.backward()
    optimizer.step()

    # Store training loss
    train_losses.append(train_loss.item())

    # 4. Calculate test loss
    tri_model.eval()
    with torch.no_grad():
        test_outputs = tri_model(X_test.unsqueeze(-1))
        test_loss = criterion(test_outputs.squeeze(), Y_test)

    # Store test loss
    test_losses.append(test_loss.item())

    # Print losses for each epoch
    print(
        f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Test Loss: {test_loss.item():.4f}")

Epoch [1/100], Train Loss: 1.1729, Test Loss: 1.2888
Epoch [2/100], Train Loss: 1.0931, Test Loss: 1.2314
Epoch [3/100], Train Loss: 1.0353, Test Loss: 1.2046
Epoch [4/100], Train Loss: 0.9916, Test Loss: 1.1895
Epoch [5/100], Train Loss: 0.9566, Test Loss: 1.1783
Epoch [6/100], Train Loss: 0.9268, Test Loss: 1.1686
Epoch [7/100], Train Loss: 0.9007, Test Loss: 1.1613
Epoch [8/100], Train Loss: 0.8777, Test Loss: 1.1580
Epoch [9/100], Train Loss: 0.8577, Test Loss: 1.1593
Epoch [10/100], Train Loss: 0.8406, Test Loss: 1.1650
Epoch [11/100], Train Loss: 0.8261, Test Loss: 1.1744
Epoch [12/100], Train Loss: 0.8138, Test Loss: 1.1864
Epoch [13/100], Train Loss: 0.8034, Test Loss: 1.1999
Epoch [14/100], Train Loss: 0.7946, Test Loss: 1.2141
Epoch [15/100], Train Loss: 0.7871, Test Loss: 1.2279
Epoch [16/100], Train Loss: 0.7808, Test Loss: 1.2410
Epoch [17/100], Train Loss: 0.7755, Test Loss: 1.2532
Epoch [18/100], Train Loss: 0.7709, Test Loss: 1.2651
Epoch [19/100], Train Loss: 0.7670, T

In [15]:
result = []
# Inference on GPU
tri_model.eval()
with torch.no_grad():
    predictions = tri_model(X_test.unsqueeze(-1))

    vals = Y_test.cpu().numpy()
    preds = predictions.cpu().numpy().squeeze(axis=-1)
    triLinear_anomaly_scores = anomaly_scores(vals, preds)


# Evaluate and collect results for each phase
for scores, true_labels, phase in [
    (triLinear_anomaly_scores, true_rising_labels[idx_training:], 'rising'),
    (triLinear_anomaly_scores, true_full_labels[idx_training:],   'full'),
]:
    metrics = get_metrics(scores, true_labels)
    metrics.update({
        'detector_name':  "TriLinear",
        'detector_type':  'semisupervised',
        'file_name':      mdwarfs_filename,
        'anomaly_phase':  phase,
        'flare_type': 'Mdwarfs',
    })
    result.append(metrics)

df_LC = pd.DataFrame(result).reset_index()
df_LC

D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,index,AUC-PR,AUC-ROC,VUS-PR,VUS-ROC,Standard-F1,PA-F1,Event-based-F1,R-based-F1,Affiliation-F,detector_name,detector_type,file_name,anomaly_phase,flare_type
0,0,0.056028,0.924778,0.137051,0.949385,0.180994,0.212766,0.175439,0.389140,0.959679,TriLinear,semisupervised,14_mdwarfs.csv,rising,Mdwarfs
1,1,0.163698,0.948188,0.435669,0.971421,0.398403,0.636943,0.393574,0.487065,0.968693,TriLinear,semisupervised,14_mdwarfs.csv,full,Mdwarfs
